<a href="https://colab.research.google.com/github/WILLIANSCASACOLA/Meus-Codigos/blob/main/Ler_Verbatins_ptbr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
from transformers import pipeline
import pandas as pd
import torch

In [112]:
# Verifica se há GPU disponível
device = 0 if torch.cuda.is_available() else -1

In [113]:
# Carrega pipeline com modelo multilíngue
analisador = pipeline("sentiment-analysis",
                      model="nlptown/bert-base-multilingual-uncased-sentiment",
                      device=device)

Device set to use cpu


In [114]:
# Lê os comentários
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Arquivos_Ler/Verbatins.xlsx")

In [115]:
df.head(10)

,ID,Nome,Data,aaa,Comentário
0,1,Ana P.,2024-08-14,Positivo,"O atendimento foi excelente, resolveram meu pr..."
1,2,Carlos M.,2024-11-03,Negativo,"Demora demais para aprovar o limite, muito bur..."
2,3,Juliana R.,2025-01-22,Positivo,"Gosto bastante do app, é fácil de usar e bem i..."
3,4,Marcos T.,2024-07-09,Negativo,"Cobram muitas taxas escondidas, fiquei decepci..."
4,5,Renata L.,2025-03-18,Negativo,"Atendimento pelo chat foi péssimo, demoraram h..."
5,6,Felipe S.,2024-10-27,Positivo,"Consegui vários descontos no crédito rotativo,..."
6,7,Luciana D.,2024-12-05,Neutro,"O cartão é bom, mas o app trava bastante."
7,8,Bruno C.,2025-02-11,Positivo,Me surpreendi com os benefícios que o cartão o...
8,9,Patrícia V.,2024-09-30,Negativo,"Não consigo acessar minha fatura há dias, absu..."
9,10,Thiago N.,2025-04-02,Positivo,"Amo o programa de pontos, já troquei por duas ..."


In [116]:
# Função para aplicar análise
def analisar_sentimento(texto):
    resultado = analisador(str(texto))[0]
    label = resultado['label']
    score = resultado['score']

    # Converte label para categoria
    estrelas = int(label[0])
    if estrelas <= 2:
        categoria = "Negativo"
    elif estrelas == 3:
        categoria = "Neutro"
    else:
        categoria = "Positivo"

    return pd.Series([label, score, categoria])

In [117]:
# Aplica ao DataFrame
df[["Label_Modelo", "Confianca", "Categoria"]] = df["Comentário"].apply(analisar_sentimento)

In [118]:
df.head(10)

,ID,Nome,Data,aaa,Comentário,Label_Modelo,Confianca,Categoria
0,1,Ana P.,2024-08-14,Positivo,"O atendimento foi excelente, resolveram meu pr...",5 stars,0.553561,Positivo
1,2,Carlos M.,2024-11-03,Negativo,"Demora demais para aprovar o limite, muito bur...",2 stars,0.462444,Negativo
2,3,Juliana R.,2025-01-22,Positivo,"Gosto bastante do app, é fácil de usar e bem i...",4 stars,0.513934,Positivo
3,4,Marcos T.,2024-07-09,Negativo,"Cobram muitas taxas escondidas, fiquei decepci...",1 star,0.581100,Negativo
4,5,Renata L.,2025-03-18,Negativo,"Atendimento pelo chat foi péssimo, demoraram h...",1 star,0.625148,Negativo
5,6,Felipe S.,2024-10-27,Positivo,"Consegui vários descontos no crédito rotativo,...",4 stars,0.439305,Positivo
6,7,Luciana D.,2024-12-05,Neutro,"O cartão é bom, mas o app trava bastante.",3 stars,0.651369,Neutro
7,8,Bruno C.,2025-02-11,Positivo,Me surpreendi com os benefícios que o cartão o...,5 stars,0.500487,Positivo
8,9,Patrícia V.,2024-09-30,Negativo,"Não consigo acessar minha fatura há dias, absu...",1 star,0.864590,Negativo
9,10,Thiago N.,2025-04-02,Positivo,"Amo o programa de pontos, já troquei por duas ...",5 stars,0.377436,Positivo


In [119]:
# Exportar o resultado
df.to_csv("/content/drive/MyDrive/Colab Notebooks/Arquivos_Ler/comentarios_analisados_vader.csv", index=False)